### integration

In [1]:
import scanpy as sc 
import matplotlib
import argparse
matplotlib.use("Agg")

Duplicate key in file PosixPath('/root/miniconda3/envs/sc/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 805 ('font.family:  sans-serif')
Duplicate key in file PosixPath('/root/miniconda3/envs/sc/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 806 ('font.sans-serif: SimHei, DejaVu Sans, Bitstream Vera Sans, Computer Modern Sans Serif, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif')
Duplicate key in file PosixPath('/root/miniconda3/envs/sc/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 807 ('axes.unicode_minus: False')


In [2]:
%run scBCN.py
from scBCN import scBCNModel

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
# 解析命令行参数
method="scBCN"
parser = argparse.ArgumentParser()  # 添加多个命令行参数的定义
parser.add_argument('--dataset',type=str,required=True, help='dataset name')
parser.add_argument("--filepath",type=str,required=True,help="folder path of stored data")
parser.add_argument("--ncelltype",type=int,required=True,help="number of celltype in dataset")
parser.add_argument("--k",type=int,default=10,help="k value to calculate MNN pair")
parser.add_argument("--resolution",type=float,default=3.0,help="resolution of clustering algorithm")
parser.add_argument("--n_hvg",type=int,default=2000,help="number of highly variable genes to be selected")
parser.add_argument("--verbose",type=bool,default=True,help="print additional information")
parser.add_argument("--savedir",type=str,required=True,help="where to save data")
parser.add_argument("--save",type=bool,default=True,help="whether to save data")
print("method=",method)
args=parser.parse_args()  # 将解析结果存储在args变量中
print("dataset=",args.dataset)
dataset=args.dataset
filepath=args.filepath
ncelltype=args.ncelltype
k=args.k
resolution=args.resolution
n_hvg=args.n_hvg
verbose=args.verbose
savedir=args.savedir
save=args.save
save_figdir=savedir

method= scBCN


usage: ipykernel_launcher.py [-h] --dataset DATASET --filepath FILEPATH
                             --ncelltype NCELLTYPE [--k K]
                             [--resolution RESOLUTION] [--n_hvg N_HVG]
                             [--verbose VERBOSE] --savedir SAVEDIR
                             [--save SAVE]
ipykernel_launcher.py: error: the following arguments are required: --dataset, --filepath, --ncelltype, --savedir


SystemExit: 2

In [5]:
# 后续更改
filepath = '/root/autodl-tmp/data/sim1'
dataset = 'sim1'
save_figdir = '/root/autodl-tmp/evaluation/'
save_dir = '/root/autodl-tmp/evaluation/'

In [6]:
dataset_path=filepath+"/"+dataset+"_raw.h5ad"# 
adata_raw=sc.read(dataset_path)
sc.settings.figdir=save_figdir+dataset+"/"+method+"/"
adata_raw

AnnData object with n_obs × n_vars = 1972 × 10000
    obs: 'celltype', 'cell', 'batch', 'BATCH'

In [7]:
#from scBCN import print_dataset_information
print_dataset_information(adata_raw, batch_key="BATCH", celltype_key="celltype")

====== print brief infomation of dataset ======
====== there are 3 batchs in this dataset ======
====== there are 4 celltypes with this dataset ======


celltype,Group1,Group2,Group3,Group4,Total
BATCH,,,,,
Batch_1,29,156,187,203,575
Batch_2,29,189,195,186,599
Batch_3,0,222,293,283,798
Total,58,567,675,672,1972


In [8]:
method="scBCN"
scbcn=scBCNModel(save_dir=save_figdir+dataset+"/"+method+"/")

2025-04-23 22:12:39,798 - root - INFO - Create log file....
2025-04-23 22:12:39,798 - root - INFO - Create scBCNModel Object Done....


In [9]:
adata=scbcn.preprocess(adata_raw, resolution=3.0, n_high_var=2000)
print(adata)

2025-04-23 22:12:43,864 - root - INFO - Running preprocess() function...
2025-04-23 22:12:43,864 - root - INFO - Leiden (resolution=3.0)
2025-04-23 22:12:43,865 - root - INFO - BATCH_key=BATCH
2025-04-23 22:12:43,866 - root - INFO - Normalize counts per cell(sum=10000.0)
2025-04-23 22:12:43,895 - root - INFO - Log1p data
2025-04-23 22:12:43,937 - root - INFO - Select HVG(n_top_genes=2000)
2025-04-23 22:12:44,394 - root - INFO - Scale batch(scale_value=10.0)
2025-04-23 22:12:44,488 - root - INFO - Calculate PCA(n_comps=100)


         Falling back to preprocessing with `sc.pp.pca` and default params.


2025-04-23 22:12:53,819 - root - INFO - Apply leiden clustring(resolution=3.0)  initization
2025-04-23 22:12:53,822 - root - INFO - Number of Cluster =35
2025-04-23 22:12:53,823 - root - INFO - clusters=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
2025-04-23 22:12:53,824 - root - INFO - Preprocess Dataset Done...


AnnData object with n_obs × n_vars = 1972 × 2000
    obs: 'celltype', 'cell', 'batch', 'BATCH', 'init_cluster'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'


In [11]:
# 后续更改
n_cluster=4
scbcn.integrate(adata, batch_key="BATCH", n_cluster=n_cluster, k=10, walk_steps=20, expect_num_cluster=n_cluster)
adata.obs["cluster_celltype"]=adata.obs["reassign_cluster"].copy()

2025-04-23 22:13:13,929 - root - INFO - k=10
2025-04-23 22:13:13,930 - root - INFO - Calculate similarity of cluster with MNN
2025-04-23 22:13:13,930 - root - INFO - Appoximate calculate MNN pairs intra batch and inter batch...
2025-04-23 22:13:13,932 - root - INFO - Calculate MNN pairs intra batch and inter batch...........
2025-04-23 22:13:13,932 - root - INFO - k=10
2025-04-23 22:13:13,933 - root - INFO - Processing datasets: (0, 0) = ('Batch_1', 'Batch_1')
2025-04-23 22:13:14,074 - root - INFO - There are (39900) MNN pairs when processing (0, 0)=('Batch_1', 'Batch_1')
2025-04-23 22:13:14,075 - root - INFO - Processing datasets: (0, 1) = ('Batch_1', 'Batch_2')
2025-04-23 22:13:14,339 - root - INFO - There are (81234) MNN pairs when processing (0, 1)=('Batch_1', 'Batch_2')
2025-04-23 22:13:14,340 - root - INFO - Processing datasets: (0, 2) = ('Batch_1', 'Batch_3')
2025-04-23 22:13:14,637 - root - INFO - There are (95404) MNN pairs when processing (0, 2)=('Batch_1', 'Batch_3')
2025-04

In [12]:
sc.pp.neighbors(adata, random_state=0, use_rep="X_emb")
sc.tl.umap(adata)
sc.pl.umap(adata,color=["BATCH","celltype"], save="_"+dataset+"_"+method+"_corrected.png")

In [13]:
adata.write(save_dir+dataset+"/"+method+"/"+dataset+"_"+method+"_corrected.h5ad")
print("done")

done


### metrics output

In [14]:
import os
os.getcwd()
print(os.environ["LD_LIBRARY_PATH"])
os.environ['R_HOME'] = '/root/miniconda3/envs/sc/lib/R'
print(os.environ['R_HOME'])

/usr/local/nvidia/lib:/usr/local/nvidia/lib64
/root/miniconda3/envs/sc/lib/R


In [15]:
from metrics import evaluate_dataset

2025-04-23 22:13:28,347 - rpy2.situation - INFO - cffi mode is CFFI_MODE.ANY
2025-04-23 22:13:28,352 - rpy2.situation - INFO - R home found: /root/miniconda3/envs/sc/lib/R
2025-04-23 22:13:28,512 - rpy2.situation - INFO - R library path: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-23 22:13:28,513 - rpy2.situation - INFO - LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-23 22:13:28,516 - rpy2.rinterface_lib.embedded - INFO - Default options to initialize R: rpy2, --quiet, --no-save
2025-04-23 22:13:28,723 - rpy2.rinterface_lib.embedded - INFO - R is already initialized. No need to initialize.


In [16]:
adata_integrated = sc.read_h5ad(save_dir+dataset+"/"+method+"/"+dataset+"_"+method+"_corrected.h5ad")

In [17]:
adata_new=sc.AnnData(adata_integrated.obsm["X_emb"])
adata_new.obs=adata_integrated.obs.copy()
adata_integrated=adata_new.copy()
adata_integrated.obsm["X_emb"]=adata_integrated.X.copy()

In [18]:
adata_integrated,scBCN3_eva=evaluate_dataset(adata_integrated,method=method,n_cluster=n_cluster)
print(scBCN3_eva)

...... method=scBCN ......
ARI: 1.0
NMI: 1.0
ASW_celltype = 0.8612532317638397
BatchKL= [0.14476046]
ilisi= 2.6587165361070397
              scBCN
ARI           1.000
NMI           1.000
ASW_celltype  0.861
BatchKL       0.145
iLISI         2.659
